In [ ]:
import glob
import re

import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils import Graph

PATH = '/home/david/data/SimLArCADe/gar-data/2PSI/'

field_strength = np.array([1,10,20,30,40,50,100,200,300,400,500,600,700,800,900,1000]) # V / cm
density = 1.15e19 # cm^-3
Td = 1e-17 # 1 Td = 10^-17 V * cm^2

reducedfield_v = (field_strength / density) / Td

In [ ]:
from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

def FitGauss(vals):
    variance = np.std(vals)
    mean = np.mean(vals)
    BINS = np.linspace(mean-2*variance,mean+2*variance,21)
    binned,edges = np.histogram(vals,bins=BINS)
    bcenters = 0.5*(edges[1:]+edges[:-1])
    guess = [mean,variance/2.,np.max(binned)]
    try:
        popt,popv = curve_fit(gauss,bcenters,binned,p0=guess)
        return popt[0],popt[1],popt[2],1
    except:
        return mean,variance,1,0
    #pope = np.sqrt(np.diag(popv))

In [ ]:
gardiff = open('lxcat/gar_diffusion.csv','r')
measurements_v = []
EN_v = []
diff_v = []
for line in gardiff:
    words = line.split()
    if (len(words) < 2): 
        if ( (len(EN_v) == len(diff_v)) and (len(EN_v) > 0) ):
            measurements_v.append([EN_v,diff_v])
        EN_v = []
        diff_v = []
    else:
        EN_v.append(float(words[0]))
        diff_v.append(float(words[1]))

In [ ]:
fig = plt.figure(figsize=(6,6))
for measurement in measurements_v:
    plt.plot(measurement[0],measurement[1],'o--')
plt.xlabel('Reduced Electric Field E/N in Td')
plt.ylabel(r'Diffusion $\times$ gas density [meter $\times$ sec. ]${}^{-1}$')
plt.xscale('log')
plt.grid()
plt.show()

The below two cells load the necessary files and compute the diffusion coefficients. At various fields, $100$ electrons were spawned and allowed to propagate for $10$ ns.

In [ ]:
file_tree = {}

for field in field_strength:
    for file in glob.glob(f'{PATH}/{field}/*.txt'):
        key = int(re.search(r'(\d*)(?:V)', file).group(1))
        if (key in file_tree):
            file_tree[key].append(file)
        else:
            file_tree[key] = [file]

The below shows the estimation process of the diffusion coefficients: the variance of each coordinate of the electrons in the cloud is plotted as a function of time. The diffusion coefficients are the slopes of these lines.

In [ ]:
for key, value in sorted(file_tree.items()):
    t = []
    D_L = []
    D_Ty = []
    D_Tz = []
    
    group = Graph(value)
    
    min_length = np.inf
    
    for i in range(group.n):
        if len(group.t[i]) < min_length:
            min_length = len(group.t[i])
            
    indices = np.arange(0, min_length, 3)

    for i in indices:
        t_avg = np.mean([k[i] for k in group.t]) * 1e-9

        x_vals = [k[i] * 1e-8 for k in group.x]
        y_vals = [k[i] * 1e-8 for k in group.y]
        z_vals = [k[i] * 1e-8 for k in group.z]
        
        t.append(t_avg)

        b,this_D_L,c,s = FitGauss(x_vals)# np.var(x_vals, ddof=1)
        D_L.append(this_D_L)

        b,this_D_Ty,c,s = FitGauss(y_vals) #np.var(y_vals, ddof=1)
        D_Ty.append(this_D_Ty)

        b,this_D_Tz,c,s = FitGauss(z_vals) #np.var(z_vals, ddof=1)
        D_Tz.append(this_D_Tz)
        
    # plt.rcParams['font.size'] = 14
    # plt.rcParams['font.family'] = 'serif'
    # fig = plt.figure(figsize=(6, 6))
    
    plt.plot(t, D_L, alpha=0.7, label='$\sigma_L^2$')
    plt.plot(t, D_Ty, alpha=0.7, label='$\sigma_{T_y}^2$')
    plt.plot(t, D_Tz, alpha=0.7, label='$\sigma_{T_z}^2$')
    
    slope_L, intercept_L, r_value, p_value, std_err = stats.linregress(t, D_L)
    slope_Ty, intercept_Ty, r_value, p_value, std_err = stats.linregress(t, D_Ty)
    slope_Tz, intercept_Tz, r_value, p_value, std_err = stats.linregress(t, D_Tz)
    
    x = np.linspace(0, 2e-7, 10)
    
    plt.plot(x, slope_L * x + intercept_L, 'C0--')
    plt.plot(x, slope_Ty * x + intercept_Ty, 'C1--')
    plt.plot(x, slope_Tz * x + intercept_Tz, 'C2--')
    
    plt.xlabel('Time [s]')
    plt.ylabel('Variance of Coords. [cm$^2$]')
    
    plt.title(f'Diff. for Field of {key} V/cm')
    
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # plt.savefig(f'lar_diffusion_at_{key}.pdf', bbox_inches='tight')

In [ ]:
threshold = 3 # Throw out points more than n standard deviations above the mean

# in cm^2 / s
D_L_all = []
D_Ty_all = []
D_Tz_all = []

# diffusion coeff. times density in [m * s]^-1
field_L = []
DN_L_all = []
field_Ty = []
DN_Ty_all = []
field_Tz = []
DN_Tz_all = []

ctr = 0
for key, value in sorted(file_tree.items()):
    group = Graph(value)

    t_avg = np.mean([k[-1] for k in group.t])

    x_vals = np.array([k[-1] for k in group.x])
    y_vals = np.array([k[-1] for k in group.y])
    z_vals = np.array([k[-1] for k in group.z])

    b,this_D_L,c,s = FitGauss(x_vals) #np.var([x_vals[k] for k in np.where(np.abs(stats.zscore(x_vals)) < threshold)[0]], ddof=1) / t_avg * 1e1
    D_L_all.append(this_D_L)
    m = (this_D_L * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
    if ( (s == 1) and (m > 1e-3)):
        DN_L_all.append( m * density * (1e6) )
        field_L.append(reducedfield_v[ctr])
    #else:
    #    DN_L_all.append(0)
    
    b,this_D_Ty,c,s = FitGauss(y_vals) #np.var([y_vals[k] for k in np.where(np.abs(stats.zscore(y_vals)) < threshold)[0]], ddof=1) / t_avg * 1e1
    D_Ty_all.append(this_D_Ty)
    m = (this_D_Ty * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
    #print (m)
    if ( (s == 1) and (m > 1e-3)):
        DN_Ty_all.append( m * density * (1e6) )
        field_Ty.append(reducedfield_v[ctr])
    #else:
    #    DN_Ty_all.append(0)
    
    b,this_D_Tz,c,s = FitGauss(z_vals)# np.var([z_vals[k] for k in np.where(np.abs(stats.zscore(z_vals)) < threshold)[0]], ddof=1) / t_avg * 1e1
    D_Tz_all.append(this_D_Tz)
    m = (this_D_Tz * 1e-6)**2 / ( 2 * (t_avg * 1e-9) )
    if ( (s == 1) and (m > 1e-3)):
        DN_Tz_all.append( m * density * (1e6) )
        field_Tz.append(reducedfield_v[ctr])
    #else:
    #    DN_Tz_all.append(0)
    ctr += 1

The cell below plots the diffusion coefficients as a function of bulk field strength. The data refers to https://lar.bnl.gov/properties/index.html#e-trans.

In [ ]:
# plt.rcParams['font.size'] = 14
# plt.rcParams['font.family'] = 'serif'
fig = plt.figure(figsize=(6, 6))

plt.plot(field_L , DN_L_all, label='$D_L$')
plt.plot(field_Ty, DN_Ty_all, label='$D_{T_y}$')
plt.plot(field_Tz, DN_Tz_all, label='$D_{T_z}$')
#plt.plot(field_strength, [4.8078, 5.2069, 5.7826, 6.3684, 6.8223, 6.9404, 6.5406, 5.8135, 5.2090, 4.4562, 3.9163, 3.5097, 3.1910, 2.9333, 2.7199], label='$D_L$ from [6]')
#plt.plot(field_strength, [4.8629, 5.5250, 6.9602, 9.4303, 13.1586, 17.2051, 19.1958, 18.1949, 16.5709, 14.2722, 12.5855, 11.3328, 10.3730, 9.6162, 9.0048], label='$D_T$ from [6]')

plt.xlabel('Reduced Electric Field E/N in Td')
plt.ylabel(r'Diffusion $\times$ gas density [m $\times$ s]${}^{-1}$')
plt.title('Diffusion in LAr')

plt.legend(loc=0)
plt.xscale('log')
plt.grid()
plt.tight_layout()
plt.show()

# plt.savefig('lar_diffusion.pdf', bbox_inches='tight')

For our own purposes, we display scatter plots and histograms of snapshots of our simulated electrons.

In [ ]:
i = 0

for key, value in sorted(file_tree.items()):
    group = Graph(value)
    min_length = np.amin([len(k) for k in group.t])

    for k in range(0, min_length, round(len(group.t[0]) / 5)):
        x = []
        y = []
        z = []
        for j in range(group.n):
            x.append(group.x[j][k])
            y.append(group.y[j][k])
            z.append(group.z[j][k])

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))

        plots = [x, y, z]
        labels = ['x', 'y', 'z']
        
        plt.grid()

        for j, subplot in enumerate([ax1, ax2, ax3]):
            subplot.grid()
            variance = np.std(plots[j])
            mean = np.mean(plots[j])
            BINS = np.linspace(mean-2*variance,mean+2*variance,21)            
            subplot.hist(plots[j], bins=BINS, color=f'C{i}')
            fitmean,fitsigma,A,s = FitGauss(plots[j])
            if (s == 1):
                xvals = np.linspace(fitmean-2*fitsigma,fitmean+2*fitsigma,40)
                subplot.plot(xvals,gauss(xvals,fitmean,fitsigma,A),'r--')
            subplot.set_xlabel(labels[j] + ' [$\mu$m]')
            subplot.set_ylabel('Instances')
            subplot.set_title(f'Coord. Dist. at {group.t[0][k]:.2f} ns at {key} V/cm')

        plt.grid()
        plt.tight_layout()
        plt.show()

    i += 1
    if i > 9:
        i = 0